In [ ]:
# | default_exp layers/codebook

# Imports

In [ ]:
# | export


import torch
from einops import rearrange
from torch import nn

from vision_architectures.utils.custom_base_model import CustomBaseModel

# Config

In [ ]:
# | export


class CodebookConfig(CustomBaseModel):
    num_vectors: int
    dim: int

    revive_dead_vectors_after_n_steps: int = 100  # 0 means never revive

# Codebook

In [ ]:
# | export


class Codebook(nn.Module):
    def __init__(self, config: CodebookConfig = {}, **kwargs):
        super().__init__()

        self.config = CodebookConfig.model_validate(config | kwargs)

        num_vectors = self.config.num_vectors
        dim = self.config.dim

        self.vectors = nn.Embedding(num_vectors, dim)

        usage_counter = torch.zeros(num_vectors)  # +ve => used for the past n steps, -ve => unused for the past n steps
        self.register_buffer("usage_counter", usage_counter, persistent=False)
        self.usage_counter: torch.Tensor  # For hinting

    def quantize(self, x: torch.Tensor, channels_first: bool = None):
        #  If channels_first is None: x: (B, T, C) if ndim == 3, else (B, C, ...)
        #  If channels_first is True: x: (B, C, ...)
        #  If channels_first is False: x: (B, ..., C)

        shape = x.shape
        ndim = x.ndim
        B = shape[0]

        if channels_first is None:
            if ndim == 3:
                channels_first = False
            else:
                channels_first = True

        if channels_first:
            forward_pattern = "b c ... -> (b ...) c"
            backward_pattern = "(b s) c -> b c s"  # s stands for flattened spatial dimensions
        else:
            forward_pattern = "b ... c -> (b ...) c"
            backward_pattern = "(b s) c -> b s c"

        # Flatten input
        x = rearrange(x, forward_pattern)

        # Compute distances
        distances = torch.cdist(x, self.vectors.weight)

        # Find nearest vectors
        indices = torch.argmin(distances, dim=1)

        # Quantize and allow gradients to propogate
        z = self.vectors(indices)
        z = x + (z - x).detach()

        # Update counters
        used_vector_indices = torch.unique(indices)
        used_vector_mask = torch.scatter(
            torch.zeros_like(self.usage_counter, dtype=torch.bool), 0, used_vector_indices, True
        )
        max_value = 2**16
        self.usage_counter[used_vector_mask] = self.usage_counter[used_vector_mask] + 1
        self.usage_counter[used_vector_mask] = self.usage_counter[used_vector_mask].clamp(min=1, max=max_value)
        self.usage_counter[~used_vector_mask] = self.usage_counter[~used_vector_mask] - 1
        self.usage_counter[~used_vector_mask] = self.usage_counter[~used_vector_mask].clamp(min=-max_value, max=-1)

        # Loss calculations
        codebook_loss = torch.mean((z.detach() - x) ** 2)
        commitment_loss = torch.mean((z - x.detach()) ** 2)

        # Return back to original shape
        z = rearrange(z, backward_pattern, b=B).reshape(shape)

        # Calculate perplexity
        with torch.no_grad():
            encodings = torch.zeros(x.shape[0], self.config.num_vectors, device=x.device)
            encodings.scatter_(1, indices.unsqueeze(1), 1)
            avg_probs = encodings.float().mean(0)
            perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))

        return z, codebook_loss, commitment_loss, perplexity

    def revive_dead_vectors(self):
        revive_vector_mask = self.usage_counter <= -self.config.revive_dead_vectors_after_n_steps
        if not revive_vector_mask.any():
            return

        revive_vectors_shape = self.vectors.weight[revive_vector_mask].shape
        num_revive_vectors = revive_vectors_shape[0]

        # Sample commonly used vectors from the codebook
        sampling_probabilities = self.usage_counter.clone()
        sampling_probabilities = (
            sampling_probabilities - sampling_probabilities[~revive_vector_mask].min() + 1e-9
        )  # Make all positive, avoid all being 0
        sampling_probabilities[revive_vector_mask] = 0  # Don't sample dead vectors
        selected_vectors_mask = torch.multinomial(sampling_probabilities, num_revive_vectors, replacement=True)
        selected_vectors = self.vectors(selected_vectors_mask).detach()

        # Add noise to the selected vectors
        with torch.no_grad():
            std = self._estimate_codebook_distance() * 0.1
        noised_selected_vectors = selected_vectors + torch.randn_like(selected_vectors) * std

        # Replace dead vectors with noised selected vectors
        self.vectors.weight.data[revive_vector_mask] = noised_selected_vectors

        self.usage_counter[revive_vector_mask] = 0

    def forward(self, x: torch.Tensor, channels_first: bool = None):
        quantized_output = self.quantize(x, channels_first)
        if self.config.revive_dead_vectors_after_n_steps > 0:
            self.revive_dead_vectors()
        return quantized_output

    def _estimate_codebook_distance(self, max_sample=500):
        """Estimate mean distance between codebook vectors"""
        with torch.no_grad():
            if self.vectors.weight.shape[0] > max_sample:
                # Sample a subset for efficiency
                idx = torch.randperm(self.vectors.weight.shape[0])[:100]
                vectors_weight = self.vectors.weight[idx]
            else:
                vectors_weight = self.vectors.weight

            distances = torch.cdist(vectors_weight, vectors_weight)
            mask = ~torch.eye(distances.shape[0], dtype=torch.bool, device=distances.device)  # Exclude self-distances
            codebook_distance = distances[mask].mean()

        return codebook_distance

In [ ]:
test = Codebook(num_vectors=32, dim=8, revive_dead_vectors_after_n_steps=2)
display(test)

sample_input = torch.randn(2, 2**10, 8, requires_grad=True)
output = test(sample_input)
display([output[0].shape, *output[1:]])

sample_input = torch.randn(2, 8, 2, 2, 2, requires_grad=True)
output = test(sample_input)
display(test.usage_counter)
output = test(sample_input)
display(test.usage_counter)
output = test(sample_input)
display(test.usage_counter)
display([output[0].shape, *output[1:]])


Codebook(
  (vectors): Embedding(32, 8)
)


[
    torch.Size([2, 1024, 8]),
    tensor(0.6643, grad_fn=<MeanBackward0>),
    tensor(0.6643, grad_fn=<MeanBackward0>),
    tensor(23.4808)
]


tensor([-1.,  2., -1., -1., -1.,  2.,  2., -1.,  2.,  2., -1., -1., -1., -1.,
         2.,  2.,  2., -1.,  2.,  2., -1., -1., -1., -1., -1.,  2., -1., -1.,
        -1., -1., -1., -1.])


tensor([0., 3., 0., 0., 0., 3., 3., 0., 3., 3., 0., 0., 0., 0., 3., 3., 3., 0.,
        3., 3., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0.])


tensor([-1., -1., -1.,  1., -1., -1., -1., -1., -1.,  4.,  1., -1., -1., -1.,
         4.,  4., -1., -1.,  4.,  4., -1.,  1., -1., -1.,  1.,  4.,  1., -1.,
        -1., -1.,  1., -1.])


[
    torch.Size([2, 8, 2, 2, 2]),
    tensor(0.5741, grad_fn=<MeanBackward0>),
    tensor(0.5741, grad_fn=<MeanBackward0>),
    tensor(10.9497)
]

# nbdev

In [ ]:
!nbdev_export